In [1]:
from api import *
import os

In [2]:
wallet = os.getenv("polygon_wallet")
api_key = os.getenv("moralis_api_key")
database_url = os.getenv("DATABASE_URL")
update = True
indexes_to_drop = 33

wallet_balance = get_account_balance(wallet, api_key, update)
wallet_df = wallet_data(wallet_balance, "WBTC")

INFO 19:19:24: Retrieving transactions for wallet: 0xC29729D92b9D6bb5EE2bE9dFad1E6C2A2E5EE39f
INFO 19:19:26: Retrieved 1 transactions
INFO 19:19:27: Getting token balances for block 71566909.
INFO 19:19:32: Getting token balances for block 71606890.


In [3]:
result = (
    wallet_df.reset_index()["total_usd"]
    .pct_change()
    .drop(index=indexes_to_drop)
    + 1
).cumprod() - 1

btc_result = (
    wallet_df.reset_index()["usdPrice"]
    .pct_change()
    .drop(index=indexes_to_drop)
    + 1
).cumprod() - 1

results = pd.concat(
    [result.rename("Modelo"), btc_result.rename("BTC")], axis=1
)

results = results.fillna(0)
results["highest_BTC"] = results["BTC"].cummax()
results["highest_Modelo"] = results["Modelo"].cummax()
results["data"] = wallet_df.reset_index()["blockTimestamp"]
results = results.set_index("data")

results["asset_ratio"] = (
    wallet_df.set_index("blockTimestamp")
    .loc[results.index]
    .drop_duplicates()["asset_ratio"]
)

results[results.columns[:-1]] = results[results.columns[:-1]]

In [4]:
wallet_df = wallet_data(wallet_balance, "WBTC")
wallet_df = results.copy()
wallet_df = (wallet_df.diff()).resample("M").sum() * 100
wallet_df = wallet_df.iloc[:, :2].round(2)

wallet_df.index = wallet_df.index.strftime("%Y-%m-%d")

if wallet_df.empty:
    response = jsonify({"error": "No data available"})

if update:
    database_url = os.getenv("DATABASE_URL")
    wallet_df.to_sql(
        "wallet_balance_monthly",
        con=database_url,
        if_exists="replace",
        index=True,
        index_label="date",
    )


In [5]:
database_df = pd.read_sql("btc", con=database_url, index_col="date").iloc[:-2]
database_df.to_sql(
    "btc",
    con=database_url,
    if_exists="replace",
    index=True,
    index_label="date",
)

old_database = pd.read_sql("btc", con=database_url, index_col="date")
last_time = pd.to_datetime(old_database.index[-2]).timestamp() * 1000

ccxt_api = CcxtAPI(
    "BTC/USDT",
    "1d",
    ccxt.binance(),
    since=int(last_time),
    verbose="Text",
)

new_data = ccxt_api.get_all_klines().to_OHLCV().data_frame
btc = new_data.combine_first(old_database).drop_duplicates()

btc.to_sql(
    "btc",
    con=database_url,
    if_exists="replace",
    index=True,
    index_label="date",
)

INFO 19:19:57: Starting requests
INFO 19:19:58: Qty: 5 - Total: 100% complete
INFO 19:19:58: Qty: 5 - Total: 100.00% complete
INFO 19:19:58: Requests elapsed time: 0.4182723000012629



883

In [6]:
model_pipeline = ModelPipeline("btc", database_url=database_url)
recommendations = model_pipeline.get_model_recommendations()

recommendations.to_sql(
    "model_recommendations",
    con=database_url,
    if_exists="replace",
    index=True,
    index_label="date",
)

245